In [1]:
from pathlib import Path
import datetime
from zoneinfo import ZoneInfo
from datetime import datetime
import os
import scipy.io as sio
import numpy as np
import pandas as pd

import pynwb
from pynwb import NWBHDF5IO, NWBFile

import manimoh_utils as mu
import manimoh_nwb_converters as mnc

In [2]:
# Create an NWB file
from datetime import datetime
from dateutil.tz import tzlocal
session_dir = 'E:\\odor-pixels\\M541-2024-08-31'
session_metadata = mu.parse_expkeys(session_dir)
out_nwb = NWBFile(
    session_description="my first synthetic recording", #TODO: Can this be gleaned from exp keys?
    identifier='-'.join([session_metadata['subject'], session_metadata['date']]),
    session_start_time=datetime.now(tzlocal()), #TODO: Get from .meta file
    experimenter=[
        session_metadata['experimenter']
    ],
    lab="vandermeerlab",
    institution="Dartmouth College",
    experiment_description="Head-fixed mouse presented with odor sequences",
    keywords=["ecephys", "exploration", "wanderlust"], #TODO: Needs editing
    )

In [3]:
# Add LFP electrodes and devices
device_labels = []
if os.path.exists(session_dir + "//imec0_clean_lfp.mat"):
    device_labels.append("imec0")
if os.path.exists(session_dir + "//imec1_clean_lfp.mat"):
    device_labels.append("imec1")
# add LFP electrodes table to nwb file
mnc.add_lfp_electrodes_to_nwb(session_dir, out_nwb, session_metadata, device_labels)
# add LFP traces to nwb file
mnc.add_lfp_data_to_nwb(session_dir, out_nwb, session_metadata, device_labels)

In [4]:
# Add sorting electrodes and devices
device_labels = []
if os.path.exists(session_dir + "//clean_units_imec0.mat"):
    device_labels.append("imec0")
if os.path.exists(session_dir + "//clean_units_imec1.mat"):
    device_labels.append("imec1")
mnc.add_sorting_electrodes_to_nwb(session_dir, out_nwb, session_metadata, device_labels)
# add spike times, waveforms, and other information to nwb file
mnc.add_sorting_data_to_nwb(session_dir, out_nwb, session_metadata, device_labels)

In [5]:
# Add behavioral epoohs
mnc.add_intervals_to_nwb(session_dir, out_nwb, session_metadata)

### Write the actual NWB File

In [6]:
io = NWBHDF5IO("E:\\odor-pixels\\M541-2024-08-31\\test_epochs.nwb", mode='w')
io.write(out_nwb)
io.close() # This is crtitcal and nwbinspector won't work without it